In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import import_ipynb
import DataLoader_P_speed
import utils
import time
import numpy as np

importing Jupyter notebook from DataLoader_P_speed.ipynb
importing Jupyter notebook from utils.ipynb


In [2]:
class args():
    def __init__(self):
        self.base_out_features = 2048
        self.t_obs = 8
        self.batch_size = 20
        self.scene_shape = [3, 144, 256]
        self.base_shape = [2048, 9, 16]
        self.hidden_size = 128
        self.embedding_size = 64
        self.input_size = 2
        self.output_size = 2
        self.sample = True
        self.n_train_sequences = 15000
        self.n_val_sequences = 10
        self.trainOrVal = 'train'
        args.citywalks = False
        
        #dataset
        self.jaad_dataset = '../../../../data/smailait-data/jaad/annotations'
        self.dtype = 'train'
        self.from_file = True
        self.file = '/data/smailait-data/train_speed.csv'
        self.val_file = '/data/smailait-data/val_speed.csv'
        self.seq_len = 4
        self.predicted_seq_len = 4
        self.crop = 0.3
        self.activity_h = 9
        self.activity_w = 16
        self.truncate = 0
        self.final_frame_offset = 0
        self.loader_workers = 20
        self.loader_shuffle = True
        self.pin_memory = False
        self.image_resize = [144, 256]
        self.image_size = [1080, 1920]
        self.device='cuda'
        
args = args()

In [3]:
class P_LSTM(nn.Module):
    def __init__(self):
        super(P_LSTM, self).__init__()
        
        self.pos_encoder = nn.LSTM(input_size=4, hidden_size=256)
        
        self.speed_encoder = nn.LSTM(input_size=4, hidden_size=256)
        
        self.decoder = nn.LSTMCell(input_size=4, hidden_size=256)
        
        self.out = nn.Linear(in_features=256, out_features=4)
        
        self.activation = nn.Hardtanh(min_val=-100, max_val=100)
        
    def forward(self, pos, speed):
        _, (hp, cp) = self.pos_encoder(pos.permute(1,0,2))
        _, (hs, cs) = self.speed_encoder(speed.permute(1,0,2))
        
        outputs = torch.tensor([], device='cuda')
        in_dec = speed[:,-1,:]
        
        hp = hp.squeeze(0)
        cp = cp.squeeze(0)
        hs = hs.squeeze(0)
        cs = cs.squeeze(0)
        #hd = torch.cat((hp, hs), dim=1)
        #cd = torch.cat((cp, cs), dim=1)
        hd = hp+hs
        cd = cp+hs
        for i in range(8):
            hd, cd = self.decoder(in_dec, (hd, cd))
            output = self.activation(self.out(hd))
            outputs = torch.cat((outputs, output.unsqueeze(1)), dim = 1)
            in_dec = output.detach()
        return outputs

In [4]:
net = P_LSTM().to('cuda')

In [5]:
train, val, test = DataLoader_P_speed.data_loader(args)

frame
crossing_true
imagefolderpath
scenefolderpath
filename
file
bounding_box
im_size
future_bounding_box
train  loaded
frame
crossing_true
imagefolderpath
scenefolderpath
filename
file
bounding_box
im_size
future_bounding_box
train  loaded
frame
crossing_true
imagefolderpath
scenefolderpath
filename
file
bounding_box
im_size
future_bounding_box
val  loaded


In [6]:
learning_rate = 0.0001
optimizer = optim.Adam(net.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=10, threshold = 1e-8, verbose=True)
criterion = nn.MSELoss()
train_scores = []
val_scores = []

In [7]:
for epoch in range(100):
    avg_epoch_train_loss = 0
    avg_epoch_val_loss = 0
    ade = 0
    fde = 0
    aiou = 0
    fiou = 0
    counter = 0
    for idx, (obs, target, obs_p, target_p) in enumerate(train):
        counter += 1
        obs = obs.type(torch.float32).to(device='cuda')
        target = target.type(torch.float32).to(device='cuda')
        obs_p = obs_p.type(torch.float32).to(device='cuda')
        target_p = target_p.type(torch.float32).to(device='cuda')
        
        net.zero_grad()
        preds = net(obs_p, obs)
        train_loss = criterion(preds, target)/10
        train_loss.backward()
        optimizer.step()
        
        avg_epoch_train_loss += float(train_loss)
        
    avg_epoch_train_loss/=counter
    train_scores.append(avg_epoch_train_loss)
    
    counter=0
    for idx, (obs, target, obs_p, target_p) in enumerate(test):
        counter+=1
        obs = obs.type(torch.float32).to(device='cuda')
        target = target.type(torch.float32).to(device='cuda')
        obs_p = obs_p.type(torch.float32).to(device='cuda')
        target_p = target_p.type(torch.float32).to(device='cuda')
        
        with torch.no_grad():
            preds = net(obs_p, obs)
            val_loss = criterion(preds, target)/10
            preds_p = utils.speed2pos(preds, obs_p, 20, 'cuda')
            ade += float(utils.ADE_c(preds_p, target_p[:,:-1,:]))
            fde += float(utils.FDE_c(preds_p, target_p[:,:-1,:]))
            aiou += float(utils.AIOU(preds_p, target_p[:,:-1,:]))
            fiou += float(utils.FIOU(preds_p, target_p[:,:-1,:]))
            
        avg_epoch_val_loss += float(val_loss)
        
    avg_epoch_val_loss/=counter
    val_scores.append(avg_epoch_val_loss)
    ade/=counter
    fde/=counter     
    aiou/=counter
    fiou/=counter
    
    scheduler.step(val_loss)
    
    print('e:', epoch, ' | train_loss: %.4f'% avg_epoch_train_loss, ' | val_loss: %.4f'% avg_epoch_val_loss, 
          '| ade: %.4f'% ade, '| fde: %.4f'% fde, '| aiou: %.4f'% aiou, '| fiou: %.4f'% fiou)

e: 0  | train_loss: 2.5565  | val_loss: 2.4496 | ade: 12.0830 | fde: 20.9679 | aiou: 0.7138 | fiou: 0.5709
e: 1  | train_loss: 1.7650  | val_loss: 2.1944 | ade: 10.5242 | fde: 17.9144 | aiou: 0.7268 | fiou: 0.5868
e: 2  | train_loss: 1.6114  | val_loss: 2.1044 | ade: 9.7829 | fde: 16.3339 | aiou: 0.7427 | fiou: 0.6183
e: 3  | train_loss: 1.5412  | val_loss: 2.0453 | ade: 9.6274 | fde: 16.0304 | aiou: 0.7447 | fiou: 0.6217
e: 4  | train_loss: 1.4893  | val_loss: 2.0080 | ade: 9.5139 | fde: 15.7917 | aiou: 0.7446 | fiou: 0.6213
e: 5  | train_loss: 1.4470  | val_loss: 1.9606 | ade: 9.4586 | fde: 15.7702 | aiou: 0.7439 | fiou: 0.6199
e: 6  | train_loss: 1.4079  | val_loss: 1.9218 | ade: 9.3626 | fde: 15.4703 | aiou: 0.7467 | fiou: 0.6278
e: 7  | train_loss: 1.3586  | val_loss: 1.8705 | ade: 9.4645 | fde: 15.8960 | aiou: 0.7411 | fiou: 0.6165
e: 8  | train_loss: 1.3025  | val_loss: 1.8064 | ade: 9.4806 | fde: 15.8639 | aiou: 0.7443 | fiou: 0.6247
e: 9  | train_loss: 1.2586  | val_loss: 1.75

e: 75  | train_loss: 0.8453  | val_loss: 1.7211 | ade: 9.2833 | fde: 15.5488 | aiou: 0.7481 | fiou: 0.6260
Epoch    76: reducing learning rate of group 0 to 3.1250e-06.
e: 76  | train_loss: 0.8441  | val_loss: 1.7239 | ade: 9.2358 | fde: 15.4913 | aiou: 0.7487 | fiou: 0.6259
e: 77  | train_loss: 0.8411  | val_loss: 1.7183 | ade: 9.2989 | fde: 15.6106 | aiou: 0.7483 | fiou: 0.6261
e: 78  | train_loss: 0.8405  | val_loss: 1.7288 | ade: 9.2952 | fde: 15.5928 | aiou: 0.7480 | fiou: 0.6255
e: 79  | train_loss: 0.8400  | val_loss: 1.7260 | ade: 9.2886 | fde: 15.5733 | aiou: 0.7483 | fiou: 0.6262
e: 80  | train_loss: 0.8394  | val_loss: 1.7272 | ade: 9.2877 | fde: 15.5847 | aiou: 0.7484 | fiou: 0.6257
e: 81  | train_loss: 0.8390  | val_loss: 1.7278 | ade: 9.3035 | fde: 15.6014 | aiou: 0.7479 | fiou: 0.6254
e: 82  | train_loss: 0.8385  | val_loss: 1.7246 | ade: 9.2818 | fde: 15.5957 | aiou: 0.7484 | fiou: 0.6261
e: 83  | train_loss: 0.8381  | val_loss: 1.7262 | ade: 9.2747 | fde: 15.5872 | aio

In [9]:
avg_epoch_val_loss = 0
ade = 0
fde = 0
aiou = 0
fiou = 0
counter=0
for idx, (obs, target, obs_p, target_p) in enumerate(test):
    counter+=1
    obs = obs.type(torch.float32).to(device='cuda')
    target = target.type(torch.float32).to(device='cuda')
    obs_p = obs_p.type(torch.float32).to(device='cuda')
    target_p = target_p.type(torch.float32).to(device='cuda')

    with torch.no_grad():
        preds = net(obs_p[:,1:,:], obs)
        val_loss = criterion(preds, target)/100
        preds_p = utils.speed2pos(preds, obs_p.to('cuda'), 20, 'cuda')
        ade += float(utils.ADE_c(preds_p, target_p[:,:-1,:]))
        fde += float(utils.FDE_c(preds_p, target_p[:,:-1,:]))
        aiou += float(utils.AIOU(preds_p, target_p[:,:-1,:]))
        fiou += float(utils.FIOU(preds_p, target_p[:,:-1,:]))

    avg_epoch_val_loss += float(val_loss)

avg_epoch_val_loss/=counter
ade/=counter
fde/=counter     
aiou/=counter
fiou/=counter
print('val_loss: %.4f'% avg_epoch_val_loss, '| ade: %.4f'% ade, '| fde: %.4f'% fde, '| aiou: %.4f'% aiou, '| fiou: %.4f'% fiou)

val_loss: 0.1737 | ade: 9.3038 | fde: 15.5964 | aiou: 0.7479 | fiou: 0.6256
